In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'name':['Паша', 'Дима', 'Коля', 'Саша', 'Семен'],\
                   'ages':[27,23,32,30,33],\
                   'strong_hand':['right', 'left', ['right', 'left'], 'right', ['left','right']],\
                  'work_days':[226,243,251,189,247],\
                  'lateness':[11,15,0,7,22]})

df_2 = pd.DataFrame({'name':['Аня', 'Даша', 'Леся', 'Инна', 'Маша'],\
                     'ages':[22,23,30,20,23],\
                    'work_days':[205,215,254,198,224],\
                    'lateness':[19,16,14,9,16],\
                   'weight_in_may':[52,67,58,76,49],\
                   'weight_in_november':[53,58,59,79,51]})
display(df)
display(df_2)

,name,ages,strong_hand,work_days,lateness
0,Паша,27,right,226,11
1,Дима,23,left,243,15
2,Коля,32,"[right, left]",251,0
3,Саша,30,right,189,7
4,Семен,33,"[left, right]",247,22


,name,ages,work_days,lateness,weight_in_may,weight_in_november
0,Аня,22,205,19,52,53
1,Даша,23,215,16,67,58
2,Леся,30,254,14,58,59
3,Инна,20,198,9,76,79
4,Маша,23,224,16,49,51


## Редко используемые функции

Функция для разложения списка из колонки таблицы на строки

In [3]:
df.explode('strong_hand')

,name,ages,strong_hand,work_days,lateness
0,Паша,27,right,226,11
1,Дима,23,left,243,15
2,Коля,32,right,251,0
2,Коля,32,left,251,0
3,Саша,30,right,189,7
4,Семен,33,left,247,22
4,Семен,33,right,247,22


Функция для забора данных из Гугл таблиц

In [4]:
def read_gsheet(id, list_id=None):
    link = 'https://docs.google.com/spreadsheets/d/' + id + '/export?format=csv'
    if list_id:
        link += f'&gid={list_id}'
    return pd.read_csv(link)

Добавление строки "Итого" к датафрейму

In [5]:
df.append(df.sum(numeric_only=True), ignore_index=True)

,name,ages,strong_hand,work_days,lateness
0,Паша,27.0,right,226.0,11.0
1,Дима,23.0,left,243.0,15.0
2,Коля,32.0,"[right, left]",251.0,0.0
3,Саша,30.0,right,189.0,7.0
4,Семен,33.0,"[left, right]",247.0,22.0
5,NaN,145.0,NaN,1156.0,55.0


## Использование статистических тестов

In [5]:
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats.power import zt_ind_solve_power

# функция, для подсчета времени, необходимого на тест изменения конверсии при указании абсолютного эффекта
def need_group_size_absolute(month_size, actual_conv_rate, min_effect_size, alpha, power):
    
    effect_size = proportion_effectsize(actual_conv_rate, actual_conv_rate + min_effect_size)
    # calculate the sample size for each group
    sample_size = zt_ind_solve_power(effect_size=effect_size, alpha=alpha, power=power, alternative='two-sided')
    # round the sample size to the nearest whole number
    rounded_sample_size = round(sample_size)
    days = (rounded_sample_size * 2) / (month_size / 30)
    
    print(f"""
Alpha: {alpha}
Power: {power} 
Rel. MDE(absolute): {min_effect_size:.1%} 
Group Size: {rounded_sample_size}
Estimated test duration: {days:.0f}
If no changes detected we may loose: {int(round(month_size * actual_conv_rate - (actual_conv_rate - min_effect_size * actual_conv_rate) * month_size, 0))}
""")
    
    
    # функция, для подсчета времени, необходимого на тест конверсии при указании относительного эффекта
def need_group_size_relative(month_size, actual_conv_rate, min_effect_size, alpha, power):
    control_conv_rate = actual_conv_rate
    treatment_conv_rate = control_conv_rate * (1 + min_effect_size)
    mde = (treatment_conv_rate - control_conv_rate) / control_conv_rate
    effect_size = proportion_effectsize(treatment_conv_rate, control_conv_rate)
    sample_size = zt_ind_solve_power(effect_size=effect_size, nobs1=None, alpha=alpha, power=power, ratio=1.0, alternative='two-sided')
    rounded_sample_size = round(sample_size)
    days = (rounded_sample_size * 2) / (month_size / 30)
    
    print(f"""
Alpha: {alpha}
Power: {power} 
Rel. MDE(relative): {min_effect_size:.1%} 
Group Size: {rounded_sample_size}
Estimated test duration: {days:.0f}
If no changes detected we may loose: {int(round(month_size * actual_conv_rate - (actual_conv_rate - min_effect_size * actual_conv_rate) * month_size, 0))}
""")

In [11]:
need_group_size_relative(50000, 0.2, 0.03, 0.05, 0.8)


Alpha: 0.05
Power: 0.8 
Rel. MDE(relative): 3.0% 
Group Size: 70545
Estimated test duration: 85
If no changes detected we may loose: 300



Ошибки первого и второго рода:
alpha - вероятность обнаружить различия там, где их нет (на самом деле нулевая гипотеза верна, но мы ее отвергаем)- ошибка 1 рода
beta - вероятность не обнаружить различия там, где они присутствуют (гипотеза не верна, но мы ее не отвергли)- ошибка 2 рода

In [6]:
from scipy import stats as st
alpha = 0.05

Проверка гипотезы о равенстве среднего генеральной совокупности некоторому значению

In [7]:

print('Средний возраст парней из выборки равен: {:.1f} лет'.format(df['ages'].mean()))
results = st.ttest_1samp(df['ages'], 31)
# df['ages'] - выборка
# 31 - предполагаемое среднее, на равенство которому мы делаем тест
print('p-значение: ', results.pvalue)

if results.pvalue < alpha:
    print('Отклоняем нулевую гипотезу о равенстве среднего возраста всех парней 31 году')
else:
    print('Недостаточно оснований отклонить нулевую гипотезу о равенстве среднего возраста всех парней 31 году ')

Средний возраст парней из выборки равен: 29.0 лет
p-значение:  0.3327105135083833
Недостаточно оснований отклонить нулевую гипотезу о равенстве среднего возраста всех парней 31 году 


Проверка гипотезы о равенстве среднего двух генеральных совокупностей по взятым из них выборкам

In [8]:
sample_1 = df['ages'] #- выборка из первой генеральной совокупности
sample_2 = df_2['ages'] #- выборка из второй генеральной совокупности

print('Средний возраст выборки парней: {:.1f} лет'.format(sample_1.mean()))
print('Средний возраст выборки девушек: {:.1f} лет'.format(sample_2.mean()))

results = st.ttest_ind(sample_1, sample_2, equal_var = True)
# equal_var - считать ли равными дисперсии выборок, по умолчанию имеет значение True
print('p-значение: ', results.pvalue)

if results.pvalue < alpha:
    print('Отклоняем нулевую гипотезу о равенстве среднего возраста парней и девушек')
else:
    print('Недостаточно оснований отклонить нулевую гипотезу о равенстве среднего возраста парней и девушек')

Средний возраст выборки парней: 29.0 лет
Средний возраст выборки девушек: 23.6 лет
p-значение:  0.061269663745989425
Недостаточно оснований отклонить нулевую гипотезу о равенстве среднего возраста парней и девушек


Проверка гипотезы о равенстве средних двух генеральных совокупностей для зависимых (парных) выборок

In [9]:
before = df_2['weight_in_may']
after = df_2['weight_in_november']

print('Средний вес выборки девушек в мае: {:.1f} кг'.format(before.mean()))
print('Средний вес выборки девушек в ноябре: {:.1f} кг'.format(after.mean()))

results = st.ttest_rel(before, after)
print('p-значение: ', results.pvalue)

if results.pvalue < alpha:
    print('Отклоняем нулевую гипотезу о равенстве среднего веса девушек в мае и ноябре')
else:
    print('Недостаточно оснований отклонить нулевую гипотезу о равенстве среднего веса девушек в мае и ноябре')

Средний вес выборки девушек в мае: 60.4 кг
Средний вес выборки девушек в ноябре: 60.0 кг
p-значение:  0.8634497235108648
Недостаточно оснований отклонить нулевую гипотезу о равенстве среднего веса девушек в мае и ноябре


Проверка гипотезы о равенстве конверсий

In [11]:
from statsmodels.stats.proportion import proportions_ztest

mans_lateness = df['lateness'].sum()
womans_lateness = df_2['lateness'].sum()

mans_work_days = df['work_days'].sum()
womans_work_days = df_2['work_days'].sum()

print('Парни:', mans_lateness, 'из', mans_work_days, ', конверсия в опаздания: {:.2%}'.format(mans_lateness/mans_work_days))
print('Девушки:', womans_lateness, 'из', womans_work_days, ', конверсия в опаздания: {:.2%}'.format(womans_lateness/womans_work_days))

days_with_lateness = np.array([mans_lateness, womans_lateness]) # - oпазданий у парней и опазданий у девушек
total_days = np.array([mans_work_days, womans_work_days]) # - рабочих дней у парней и девушек

stat, pvalue = proportions_ztest(days_with_lateness, total_days)

print('P-value равен: {:.5f}'.format(pvalue))

if pvalue < alpha:
    print('Отклоняем нулевую гипотезу о равенстве частоты опозданий на работу у парней и девушек, c вероятностью более 95% конверсии различаются')
else:
    print('Недостаточно оснований отклонить нулевую гипотезу о равенстве частоты опазданий парней и девушек')

Парни: 55 из 1156 , конверсия в опаздания: 4.76%
Девушки: 74 из 1096 , конверсия в опаздания: 6.75%
P-value равен: 0.04182
Отклоняем нулевую гипотезу о равенстве частоты опозданий на работу у парней и девушек, c вероятностью более 95% конверсии различаются


## Визуализация данных